In [19]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob

# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "Osiris//monitoring//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import rawEventBuilder
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
import datetime

hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest
print(data_list)
file_path = [dir_path+"//data//"+"proAnubis_240731_0217.raw"] #
file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list))[:1] # insert your file
print(file_path)

['proAnubis_240811_2240.raw', 'proAnubis_240810_2240.raw', 'proAnubis_240810_2040.raw', 'proAnubis_240810_1840.raw', 'proAnubis_240810_1640.raw', 'proAnubis_240810_1440.raw', 'proAnubis_240810_1240.raw', 'proAnubis_240810_1040.raw', 'proAnubis_240810_0840.raw', 'proAnubis_240810_0640.raw', 'proAnubis_240810_0440.raw', 'proAnubis_240810_0240.raw', 'proAnubis_240810_0040.raw']
['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw']


In [20]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)

import matplotlib.pyplot as plt
interval = 100 # Set your monitoring chunck size
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
actual_mets = {}
last_time = None
for file in file_path:
    max_process_event_chunk = 10_000 # End the loop early
    file_name = file.split("//")[-1][:-4] #must be a right file
    fReader = rawFileReader.fileReader(file) # reload in the classs object
    processedEvents = 0 # Initialisation
    mets = []
    binsx = []
    initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the initial event chunk
    originTime = min([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
    chunks = []
    with tqdm(total=max_process_event_chunk, desc=f"Processing {file_name}", unit='Events') as pbar:
        while processedEvents < max_process_event_chunk:
            processedEvents += 1
            try:
                event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the aligned events
                if not event_chunk:
                    #print("Misaligned:", processedEvents)
                    continue
            except Exception as e:
                print("Exception:", e)
                max_process_event_chunk = processedEvents
                break
            chunks.append(event_chunk)
            pbar.update(1)

    chunks = sorted(chunks, key= lambda c: (min([c[0].tdcEvents[tdc].time for tdc in range(5) if c[0].tdcEvents[tdc].time])-originTime).total_seconds())

    with tqdm(total=max_process_event_chunk, desc=f"Checking {file_name}", unit='Events') as pbar:
        for processedEvents, event_chunk in enumerate(chunks):
            event_time = min([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
            if last_time:
                if (event_time - last_time).total_seconds() > 1:
                    print("Pause:", processedEvents-1)
                    print("Event time:", (last_time - originTime).total_seconds())
                    print("Length:", (event_time - last_time).total_seconds())
                    last_event_time = min([event_chunk[-1].tdcEvents[tdc].time for tdc in range(5) if event_chunk[-1].tdcEvents[tdc].time])
                    first_event_time = min([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
                    print("Chunk size:",last_event_time - first_event_time)
            else:
                originTime = event_time 
            
            binsx.append((event_time - originTime).total_seconds())
            last_time = event_time 
            pbar.update(1)
    

Checking proAnubis_240811_2240: 100%|█████████▉| 9961/10000 [00:00<00:00, 146587.40Events/s]


In [21]:
import matplotlib.pyplot as plt
#plt.plot(sorted(check_t), check_ch)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(binsx, mets)
plt.xlabel('Time')
plt.ylabel('Counts')
plt.title('Counts vs Time')
plt.legend()
plt.grid(True)
plt.show()


ValueError: x and y must have same first dimension, but have shapes (9961,) and (0,)